In [547]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
from sklearn.compose import ColumnTransformer

In [548]:
# بارگذاری داده‌ها
Train_Data = pd.read_csv('C:\\Users\\sepeh\\PycharmProjects\\MachinLearning\\.venv\\Kaggel\\house_prices\\train.csv')
Test_Data = pd.read_csv('C:\\Users\\sepeh\\PycharmProjects\\MachinLearning\\.venv\\Kaggel\\house_prices\\test.csv')
Answer = pd.read_csv('C:\\Users\\sepeh\\PycharmProjects\\MachinLearning\\.venv\\Kaggel\\house_prices\\sample_submission.csv')


In [549]:
# محاسبه سن
Train_Data['age'] = Train_Data['YearRemodAdd'] - Train_Data['YearBuilt']
Test_Data['age'] = Test_Data['YearRemodAdd'] - Test_Data['YearBuilt']



In [550]:
# تقسیم داده‌ها به مجموعه آموزش و اعتبارسنجی
X = Train_Data[['LotArea', 'OverallQual', 'GarageCars', 'age','TotRmsAbvGrd', 'Neighborhood', 'Foundation','SaleCondition', 'HeatingQC']]
y = Train_Data['SalePrice']
print(X.head())

   LotArea  OverallQual  GarageCars  age  TotRmsAbvGrd Neighborhood  \
0     8450            7           2    0             8      CollgCr   
1     9600            6           2    0             6      Veenker   
2    11250            7           2    1             6      CollgCr   
3     9550            7           3   55             7      Crawfor   
4    14260            8           3    0             9      NoRidge   

  Foundation SaleCondition HeatingQC  
0      PConc        Normal        Ex  
1     CBlock        Normal        Ex  
2      PConc        Normal        Ex  
3     BrkTil       Abnorml        Gd  
4      PConc        Normal        Ex  


In [551]:
 # ویژگی‌های عددی و دسته‌ای
numeric_features = ['LotArea', 'OverallQual', 'GarageCars', 'age','TotRmsAbvGrd']
categorical_features = ['Neighborhood', 'Foundation','SaleCondition', 'HeatingQC']


In [552]:
# ساخت Pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), numeric_features),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_features)
    ])

In [553]:
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(min_samples_split=2, n_estimators=200, max_features=2, random_state=0))
])


In [554]:
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [555]:
model.fit(X, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  ['LotArea', 'OverallQual',
                                                   'GarageCars', 'age',
                                                   'TotRmsAbvGrd']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Neighborhood', 'Foundation',
                                                   'SaleCondition',
                                                   'HeatingQC'])])),
                ('regressor',
                 RandomForestRegressor(max_features=2, n_estimators=200,
                                       random_state=0))])

In [556]:

# پیش‌بینی و ارزیابی مدل
pred_train = model.predict(X)
# pred_val = model.predict(X_val)


In [557]:
train_rmsi = root_mean_squared_error(y, pred_train)
# val_rmsi = root_mean_squared_error(y_val, pred_val)

print('Root Mean Squared Error train: {:.2f}'.format(train_rmsi))
# print('Root Mean Squared Error val: {:.2f}'.format(val_rmsi))

Root Mean Squared Error train: 13317.12


In [558]:

# پیش‌بینی روی داده‌های تست
X_test = Test_Data[['LotArea', 'OverallQual', 'GarageCars', 'age','TotRmsAbvGrd', 'Neighborhood', 'Foundation','SaleCondition', 'HeatingQC']]


pred_test = model.predict(X_test)

In [559]:
rmse_test = root_mean_squared_error(Answer['SalePrice'], pred_test)

print(f"final rmse for test data set is {rmse_test}")

final rmse for test data set is 62982.76815083565


In [560]:
house_ids_test = Test_Data["Id"]
predictions_df = pd.DataFrame({
    'Id': house_ids_test,
    'SalePrice': pred_test
})
predictions_df.to_csv("predict_house_prices4.csv",index=False)
print(predictions_df.head())

     Id  SalePrice
0  1461  136875.50
1  1462  155514.00
2  1463  174766.31
3  1464  182453.75
4  1465  251176.00
